In [1]:
import ipywidgets as widgets

pdf_uploader = widgets.FileUpload(
    description="Original pdfs", accept=".pdf", multiple=True
)
output = widgets.Output()

In [2]:
@output.capture()
def write_pdf(contents):
    for content in contents:
        print(f"Loading {content.get('name')}...", end=" ")
        with open(content.get("name"), "wb") as f:
            f.write(content.get("content"))
        print("done")


def write_original_pdf(counter):
    if counter.get("new") == 0:
        return
    write_pdf(pdf_uploader.value)


pdf_uploader.observe(write_original_pdf, names="value")

In [3]:
def clear(_):
    output.clear_output()
    pdf_uploader.value = tuple()

In [4]:
@output.capture()
def process(_):
    import subprocess

    if len(pdf_uploader.value) == 0:
        print("Missing pdf file !")
        return

    pdf_names = {v.get("name") for v in pdf_uploader.value}
    for pdf_name in pdf_names:
        print(f"Processing {pdf_name}...", end=" ")
        result = subprocess.run(
            ["./bin/extract_rn.sh", pdf_name], capture_output=True, text=True
        )
        if result.returncode != 0:
            print("failed!")
            print("Check the following messages to know what's going on:")
            print(result.stdout)
            print(result.stderr)
        else:
            print("done")

In [5]:
@output.capture()
def download(_):
    from IPython.display import HTML

    js_code = f"""
        var a = document.createElement('a');
        a.setAttribute('download','');
        a.setAttribute('href', '../files/bin/pdf.tar.gz');
        a.click()
    """
    with output:
        display(HTML(f"<script>{js_code}</script>"))

In [6]:
process_button = widgets.Button(
    description="Process", icon="check", button_style="success"
)
download_button = widgets.Button(
    description="Download", icon="download", button_style="warning"
)
clear_button = widgets.Button(description="Clear", icon="trash", button_style="danger")
clear_button.on_click(clear)
download_button.on_click(download)
process_button.on_click(process)

top = widgets.VBox(
    [widgets.HBox([widgets.HTML(value="Upload pdf from Apogée"), pdf_uploader])],
    layout=widgets.Layout(display="flex", align_items="flex-end"),
)

bottom = widgets.HBox([process_button, download_button, clear_button])

widgets.HBox(
    [
        widgets.HTML(value="<h1>Extract RN Application</h1>"),
        top,
        widgets.HTML(value="<p style='margin-bottom:1cm;'></p>"),
        bottom,
        widgets.VBox([widgets.HTML(value="<h2>Logs</h2>"), output]),
    ],
    layout=widgets.Layout(
        display="flex", flex_flow="column", align_items="center", width="100%"
    ),
)